In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45519
2,Huelva,Confirmados PDIA 14 días,255
3,Huelva,Tasa PDIA 14 días,"49,691135491162775"
4,Huelva,Confirmados PDIA 7 días,134
5,Huelva,Tasa PDIA 7 dias,"26,112204532611024"
6,Huelva,Total Confirmados,45723
7,Huelva,Curados,44568
8,Huelva,Fallecidos,418


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45519.0


/tmp/ipykernel_9201/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12561.0


/tmp/ipykernel_9201/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_9201/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_9201/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_9201/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 7192 personas en los últimos 7 días 

Un positivo PCR cada 3127 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45519.0,134.0,255.0,513170.0,26.112205,49.691135,54.0
Huelva-Costa,26943.0,72.0,149.0,289548.0,24.866343,51.459516,28.0
Condado-Campiña,14131.0,44.0,78.0,156231.0,28.163425,49.926071,14.0
Cartaya,2000.0,23.0,40.0,20083.0,114.524722,199.173430,13.0
Sierra de Huelva-Andévalo Central,4019.0,17.0,27.0,67391.0,25.225920,40.064697,11.0
Ayamonte,2084.0,13.0,17.0,21104.0,61.599697,80.553450,7.0
Valverde del Camino,848.0,11.0,14.0,12750.0,86.274510,109.803922,7.0
Rosal de la Frontera,80.0,6.0,9.0,1697.0,353.565115,530.347672,4.0
Huelva (capital),12561.0,20.0,46.0,143837.0,13.904628,31.980645,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rosal de la Frontera,80.0,6.0,9.0,1697.0,353.565115,530.347672,4.0
Villarrasa,226.0,6.0,7.0,2211.0,271.370421,316.598824,3.0
Cartaya,2000.0,23.0,40.0,20083.0,114.524722,199.173430,13.0
Valverde del Camino,848.0,11.0,14.0,12750.0,86.274510,109.803922,7.0
Gibraleón,943.0,5.0,13.0,12737.0,39.255712,102.064850,0.0
Paterna del Campo,210.0,2.0,3.0,3457.0,57.853630,86.780445,0.0
Isla Cristina,3130.0,5.0,18.0,21393.0,23.372131,84.139672,2.0
Bonares,462.0,4.0,5.0,6060.0,66.006601,82.508251,3.0
Ayamonte,2084.0,13.0,17.0,21104.0,61.599697,80.553450,7.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aljaraque,1629.0,2.0,8.0,21474.0,9.313589,37.254354,1.0,0.250000
Punta Umbría,1265.0,1.0,4.0,15355.0,6.512537,26.050147,1.0,0.250000
Isla Cristina,3130.0,5.0,18.0,21393.0,23.372131,84.139672,2.0,0.277778
Bollullos Par del Condado,1286.0,3.0,10.0,14387.0,20.852158,69.507194,0.0,0.300000
Palos de la Frontera,1032.0,2.0,6.0,11742.0,17.032873,51.098620,2.0,0.333333
Gibraleón,943.0,5.0,13.0,12737.0,39.255712,102.064850,0.0,0.384615
Huelva (capital),12561.0,20.0,46.0,143837.0,13.904628,31.980645,4.0,0.434783
Huelva-Costa,26943.0,72.0,149.0,289548.0,24.866343,51.459516,28.0,0.483221
Rociana del Condado,667.0,1.0,2.0,7939.0,12.596045,25.192090,0.0,0.500000
